# Bundesliga Stats - Mainz vs Gladbach

Test notebook for fetching Bundesliga stats using StatsService.

**Game:** 1. FSV Mainz 05 vs Borussia Mönchengladbach

In [ ]:
# Setup
import pandas as pd
from services.stats import StatsService
from sports.futbol.bundesliga.bundesliga_config import get_bundesliga_stats_config
from sports.futbol.bundesliga.teams import TEAMS, NAME_TO_FBREF_ID, NAME_TO_SLUG

# Create StatsService for Bundesliga
config = get_bundesliga_stats_config()
service = StatsService(sport="bundesliga", config=config)

print("StatsService initialized for Bundesliga")
print(f"Rankings URL: {config.rankings_url}")
print(f"Tables to fetch: {list(config.rankings_tables.keys())}")

In [ ]:
# Team info for the game
mainz_id = NAME_TO_FBREF_ID["1. FSV Mainz 05"]
mainz_slug = NAME_TO_SLUG["1. FSV Mainz 05"]

gladbach_id = NAME_TO_FBREF_ID["Borussia Monchengladbach"]
gladbach_slug = NAME_TO_SLUG["Borussia Monchengladbach"]

print(f"Mainz: {mainz_id} / {mainz_slug}")
print(f"Gladbach: {gladbach_id} / {gladbach_slug}")

## 1. Fetch Rankings

In [ ]:
# Fetch Bundesliga rankings
print("Fetching Bundesliga rankings...")
rankings = service.fetch_rankings()

print(f"\nFetched at: {rankings.get('fetched_at')}")
print(f"Tables: {list(rankings.get('tables', {}).keys())}")

In [ ]:
# Save rankings
path = service.save_rankings(rankings)
print(f"Saved to: {path}")

In [ ]:
# Display standings table
if "standings" in rankings.get("tables", {}):
    standings_data = rankings["tables"]["standings"]["data"]
    df = pd.DataFrame(standings_data)
    print("\nBundesliga Standings:")
    display(df[["Rk", "Squad", "MP", "W", "D", "L", "GF", "GA", "GD", "Pts"]].head(18))

## 2. Fetch Mainz Profile

In [ ]:
# Fetch Mainz team profile
print(f"Fetching Mainz profile ({mainz_id})...")
mainz_profile = service.fetch_team_profile(mainz_id)

print(f"\nFetched at: {mainz_profile.get('fetched_at')}")
print(f"Tables: {list(mainz_profile.get('tables', {}).keys())}")

In [ ]:
# Save Mainz profile
path = service.save_team_profile(mainz_profile, "mainz")
print(f"Saved to: {path}")

In [ ]:
# Display Mainz fixtures
if "fixtures" in mainz_profile.get("tables", {}):
    fixtures_data = mainz_profile["tables"]["fixtures"]["data"]
    df = pd.DataFrame(fixtures_data)
    print("\nMainz Recent Fixtures:")
    cols = [c for c in ["Date", "Venue", "Result", "GF", "GA", "Opponent"] if c in df.columns]
    display(df[cols].tail(10))

## 3. Fetch Gladbach Profile

In [ ]:
# Fetch Gladbach team profile
print(f"Fetching Gladbach profile ({gladbach_id})...")
gladbach_profile = service.fetch_team_profile(gladbach_id)

print(f"\nFetched at: {gladbach_profile.get('fetched_at')}")
print(f"Tables: {list(gladbach_profile.get('tables', {}).keys())}")

In [ ]:
# Save Gladbach profile
path = service.save_team_profile(gladbach_profile, "gladbach")
print(f"Saved to: {path}")

In [ ]:
# Display Gladbach fixtures
if "fixtures" in gladbach_profile.get("tables", {}):
    fixtures_data = gladbach_profile["tables"]["fixtures"]["data"]
    df = pd.DataFrame(fixtures_data)
    print("\nGladbach Recent Fixtures:")
    cols = [c for c in ["Date", "Venue", "Result", "GF", "GA", "Opponent"] if c in df.columns]
    display(df[cols].tail(10))

## 4. Summary

In [ ]:
# Summary
print("="*60)
print("MAINZ vs GLADBACH - Stats Summary")
print("="*60)

# Find teams in standings
if "standings" in rankings.get("tables", {}):
    standings_data = rankings["tables"]["standings"]["data"]
    
    for team_name in ["Mainz 05", "Monchengladbach", "M'gladbach"]:
        for row in standings_data:
            squad = row.get("Squad", "")
            if team_name.lower() in squad.lower():
                print(f"\n{squad}:")
                print(f"  Position: {row.get('Rk')}")
                print(f"  Played: {row.get('MP')} | W: {row.get('W')} | D: {row.get('D')} | L: {row.get('L')}")
                print(f"  GF: {row.get('GF')} | GA: {row.get('GA')} | GD: {row.get('GD')}")
                print(f"  Points: {row.get('Pts')}")
                break

print("\n" + "="*60)
print("Data saved to sports/futbol/bundesliga/data/")